# SCC-DFTB repulsive potential generation (2-body)

Input: DFT_DB.db DFTB_DB.db 

Output: CCS parameters to be used in conjunction with the SK-tables generated in gen_SK.ipynb and CCS_DB.db 



In [1]:
import os
import sys
import warnings
base_dir=os.getcwd()
os.chdir(base_dir)
print('Base dir is:', base_dir)
warnings.filterwarnings('ignore')
sys.path.append('../pgm/CCS/')
sys.path.append('../pgm/python_scripts')
import json
import itertools as it
from collections import OrderedDict, defaultdict
import numpy as np
from ase import Atoms
from ase import io
from ase.calculators.neighborlist import NeighborList
import ase.db as db
from ccs.common.exceptions import ScriptError 
from ccs.fitting.main import twp_fit
from ccs.ase_tools.ccs_ase_calculator import CCS
from rep_parautomatik import pair_dist, generate_ccsdata,generate_ccsdb, write_input, fit_ccs



Base dir is: /home/broqvist/Desktop/parautomatik/EC


In [ ]:
#generate data (once, takes time) generates structures.json
cwd = os.getcwd()
if not os.path.isdir(cwd+"/CCS/"):
    os.mkdir(cwd+"/CCS/")

os.chdir(cwd+"/CCS/")
DFTB_DB=db.connect(base_dir+"/DFTB_DB.db")   
DFT_DB=db.connect(base_dir+"/DFT_DB.db")   
generate_ccsdata(DFT_DB,DFTB_DB)
os.chdir(base_dir) 

In [3]:
#input for fitting data in json format

DFT_DB=db.connect("DFT_DB.db")
elements=[]
for row in DFT_DB.select('1<id<10'):
    structure=row.toatoms()
    a=structure.get_chemical_symbols()
    for i in a: 
        if i not in elements:
            elements.append(i)
            
os.chdir(base_dir+'/CCS')            
#elements  = 'H,Li,C,O,P,Si' # ["C", "H", "Li", "O"]
print(elements)
input = {
"Onebody":elements,
"Reference":"structures.json",
"General":{
	"interface":"DFTB",
}
}

write_input(input)
os.chdir(base_dir) 

['C', 'H', 'O']


In [4]:
os.chdir(base_dir+"/CCS/")
class Args:
    cmdlineargs=None
    input = 'input.json'

args=Args()
fit_ccs(args)
os.chdir(base_dir) 

input.json /home/broqvist/Desktop/parautomatik/EC/CCS
Generating two-body potentials from one-body information.
Adding pair: C-C
Adding pair: C-H
Adding pair: C-O
Adding pair: H-H
Adding pair: H-O
Adding pair: O-O
There is linear dependence in stochiometry matrix!
    removing onebody term: H
There is linear dependence in stochiometry matrix!
    removing onebody term: O
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'C-C.spl'
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'C-H.spl'
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'C-O.spl'
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'H-H.spl'
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'H-O.spl'
Exponentail head -> 'headfit.dat'
Repulsive in Spline format -> 'O-O.spl'


# Analysis
![title](CCS/CCS_fitting_summary.png)


In [7]:
os.chdir(base_dir)

struct=db.connect("DFTB_DB.db")
db_ccs=db.connect("CCS_DB.db")

os.chdir(base_dir+"/CCS/")
with open('CCS_params.json', 'r') as f:
    CCS_params = json.load(f)

for row in struct.select():  
    structure=row.toatoms()
    structure.calc=CCS(CCS_params=CCS_params)
    structure.get_potential_energy()
    structure.get_forces()
    db_ccs.write(structure,key=row.key,CCS=True)

os.chdir(base_dir)

In [5]:

!pwd

/home/broqvist/Desktop/parautomatik/EC
